In [1]:
from pathlib import Path
import pandas as pd
from datetime import  datetime, timedelta
import numpy as np
from tqdm.notebook import tqdm 

In [2]:
# Set the directory
#Run this first for OPI cleaning
# OPI CLEANING AND AVERAGING
path = Path('U:') / 'Opi system file' / 'SQL database'/ 'New Data Processing'

In [3]:
def combine_table(df1, df1_time, df2, df2_time):
    '''
        df1 => main table
        df1_time => name of df1's time column
        df2 => table to be conmbined
        df2_time => name of df2's time column
    '''
    
    df = df1.copy()
    df['date1'] = df[df1_time].apply(lambda x: x.date())
    df['hour1'] = df[df1_time].apply(lambda x: x.hour)
        
    add_df = df2.copy()
    add_df['date2'] = add_df[df2_time].apply(lambda x: x.date())
    add_df['hour2'] = add_df[df2_time].apply(lambda x: x.hour)
    
    if (df[df1_time].min() > add_df[df2_time].min()):
        start_time = add_df[df2_time].min()
    else:
        start_time = df[df1_time].min()
    
    if (df[df1_time].max() < add_df[df2_time].max()):
        end_time = add_df[df2_time].max()
    else:
        end_time = df[df1_time].max()

    timestamps = []
    time = start_time
    while time <= end_time + timedelta(hours=1):
        timestamps.append(time)
        time = time + timedelta(hours=1)
    
    if df1_time == 'Timestamp':
        df = df.drop(['Timestamp'], axis=1)
            
    final_df = pd.DataFrame(timestamps, columns=['Timestamp'])
    final_df['date'] = final_df['Timestamp'].apply(lambda x: x.date())
    final_df['hour'] = final_df['Timestamp'].apply(lambda x: x.hour)
    
    for column in df.columns:
        final_df[column] = np.nan
        
    for column in add_df.columns:
        final_df[column] = np.nan
        
    for index, row in tqdm(final_df.iterrows(), total=len(final_df)):
        search_row = df.loc[(df['date1'] == row.date) & (df['hour1'] == row.hour)]
        if len(search_row == 1): 
            for column in df.columns:
                final_df.at[index , column] = df.at[search_row.index[0], column]
        
        search_row = add_df.loc[(add_df['date2'] == row.date) & (add_df['hour2'] == row.hour)]
        if len(search_row == 1): 
            for column in add_df.columns:
                final_df.at[index , column] = add_df.at[search_row.index[0], column]
    
    final_df = final_df.drop(['date', 'hour', 'date1', 'hour1', 'date2', 'hour2'], axis=1)
               
    return final_df.copy()
    
    
    

In [4]:
# Original file name
# filename = path / 'PHERC 11May2021.xls'
# df = pd.read_excel(filename)
# df
# df = pd.ExcelFile ('U:/Opi system file/SQL database/New Data Processing/BIN5 OPI.xlsx')
# df = pd.read_excel(xls, '_B5')
df = pd.read_excel ('U:/Opi system file/SQL database/New Data Processing/BIN5 OPI.xlsx')
df

,OPI_DATE,MC1_Avg_5_sensors
0,2020-11-03 17:51:00,14.682
1,2020-11-03 18:23:00,15.056
2,2020-11-03 18:38:00,15.222
3,2020-11-03 18:38:00,15.222
4,2020-11-03 18:45:00,14.850
...,...,...
4267,2021-06-23 06:16:00,14.286
4268,2021-06-23 07:16:00,14.118
4269,2021-06-23 08:16:00,14.138
4270,2021-06-23 09:16:00,14.164


In [5]:
current_time = df.loc[df.index[0]]['OPI_DATE']
for index, row in df.iterrows():
    timestamp = row['OPI_DATE']
    if (timestamp - current_time) > timedelta(hours=1) or index == df.index[0]:
        current_time = timestamp
    else:
        df = df.drop(index)
df

,OPI_DATE,MC1_Avg_5_sensors
0,2020-11-03 17:51:00,14.682
6,2020-11-04 12:28:00,13.218
12,2020-11-04 13:29:00,13.668
17,2020-11-04 14:44:00,13.928
22,2020-11-04 15:59:00,14.638
...,...,...
4263,2021-06-23 02:16:00,14.562
4265,2021-06-23 04:16:00,14.394
4267,2021-06-23 06:16:00,14.286
4269,2021-06-23 08:16:00,14.138


In [6]:
filename = path / 'Bin 5 headspace.xlsx'
df2 = pd.read_excel(filename)
df2

FileNotFoundError: [Errno 2] No such file or directory: 'U:Opi system file\\SQL database\\New Data Processing\\Bin 5 headspace.xlsx'

In [ ]:
headspace = df2[['Date', 'Temp_Headspace', 'RH_Headspace']].copy()
headspace = headspace.dropna()
headspace

,Date,Temp_Headspace,RH_Headspace
0,2020-11-10 02:34:00,57.2112,74.6334
1,2020-11-10 03:33:00,57.2112,74.6334
2,2020-11-10 04:34:00,57.2112,74.6334
3,2020-11-10 05:34:00,57.2112,74.6334
4,2020-11-10 06:34:00,57.2112,74.6334
...,...,...,...
5293,2021-06-21 23:20:49,78.9562,50.7637
5294,2021-06-22 00:20:49,74.4188,53.5851
5295,2021-06-22 01:21:38,73.3517,54.7692
5296,2021-06-22 02:21:37,67.7713,56.3958


In [ ]:
result = combine_table(df, 'OPI_DATE', headspace, 'Date')
result

  0%|          | 0/5562 [00:00<?, ?it/s]

,Timestamp,OPI_DATE,MC1_Avg_5_sensors,Date,Temp_Headspace,RH_Headspace
0,2020-11-03 17:51:00,2020-11-03 17:51:00,14.682,NaN,NaN,NaN
1,2020-11-03 18:51:00,NaN,NaN,NaN,NaN,NaN
2,2020-11-03 19:51:00,NaN,NaN,NaN,NaN,NaN
3,2020-11-03 20:51:00,NaN,NaN,NaN,NaN,NaN
4,2020-11-03 21:51:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
5557,2021-06-23 06:51:00,2021-06-23 06:16:00,14.286,NaN,NaN,NaN
5558,2021-06-23 07:51:00,NaN,NaN,NaN,NaN,NaN
5559,2021-06-23 08:51:00,2021-06-23 08:16:00,14.138,NaN,NaN,NaN
5560,2021-06-23 09:51:00,NaN,NaN,NaN,NaN,NaN


In [ ]:
ambient = df2[['Date_Ambient', 'Temp_Ambient ', 'CO2_Headspace']].copy()
ambient = ambient.dropna()
ambient

,Date_Ambient,Temp_Ambient,CO2_Headspace
0,2020-11-10 03:00:00.000,65.00,537.0
1,2020-11-10 04:00:00.010,64.00,537.0
2,2020-11-10 05:00:00.015,63.00,537.0
3,2020-11-10 06:00:00.020,62.00,537.0
4,2020-11-10 07:00:00.025,63.00,537.0
...,...,...,...
5293,2021-06-19 00:00:00.000,76.38,491.0
5294,2021-06-19 01:00:00.000,78.15,499.0
5295,2021-06-19 02:00:00.000,75.19,471.0
5296,2021-06-19 03:00:00.000,73.28,476.0


In [ ]:
result = combine_table(result, 'Timestamp', ambient, 'Date_Ambient')
result

  0%|          | 0/5563 [00:00<?, ?it/s]

,Timestamp,OPI_DATE,MC1_Avg_5_sensors,Date,Temp_Headspace,RH_Headspace,Date_Ambient,Temp_Ambient,CO2_Headspace
0,2020-11-03 17:51:00,2020-11-03 17:51:00,14.682,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-11-03 18:51:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-11-03 19:51:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-11-03 20:51:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-11-03 21:51:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
5558,2021-06-23 07:51:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5559,2021-06-23 08:51:00,2021-06-23 08:16:00,14.138,NaN,NaN,NaN,NaN,NaN,NaN
5560,2021-06-23 09:51:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5561,2021-06-23 10:51:00,2021-06-23 10:16:00,14.376,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
RH = df2[['RH_Amb', 'Unnamed: 7']].copy()
RH = RH.dropna()
RH

,RH_Amb,Unnamed: 7
2314,2021-02-14 13:00:00,63.58
2315,2021-02-14 14:00:00,68.48
2316,2021-02-14 15:00:00,65.96
2317,2021-02-14 16:00:00,64.40
2318,2021-02-14 17:00:00,67.91
...,...,...
5360,2021-06-21 19:00:00,61.64
5361,2021-06-21 20:00:00,56.41
5362,2021-06-21 21:00:00,56.02
5363,2021-06-21 22:00:00,69.99


In [ ]:
result = combine_table(result, 'Timestamp', RH, 'RH_Amb')
result

  0%|          | 0/5564 [00:00<?, ?it/s]

,Timestamp,OPI_DATE,MC1_Avg_5_sensors,Date,Temp_Headspace,RH_Headspace,Date_Ambient,Temp_Ambient,CO2_Headspace,RH_Amb,Unnamed: 7
0,2020-11-03 17:51:00,2020-11-03 17:51:00,14.682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-11-03 18:51:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-11-03 19:51:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-11-03 20:51:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-11-03 21:51:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5559,2021-06-23 08:51:00,2021-06-23 08:16:00,14.138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5560,2021-06-23 09:51:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5561,2021-06-23 10:51:00,2021-06-23 10:16:00,14.376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5562,2021-06-23 11:51:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
result['OPI_DATE'] = result['OPI_DATE'].apply(lambda x: x if pd.isna(x) else datetime.strftime(x, '%m/%d/%Y %I:%M %p'))
result['Date'] = result['Date'].apply(lambda x: x if pd.isna(x) else datetime.strftime(x, '%m/%d/%Y %I:%M %p'))
result['Date_Ambient'] = result['Date_Ambient'].apply(lambda x: x if pd.isna(x) else datetime.strftime(x, '%m/%d/%Y %I:%M %p'))
result = result.drop(['Timestamp'], axis=1)
result

,OPI_DATE,MC1_Avg_5_sensors,Date,Temp_Headspace,RH_Headspace,Date_Ambient,Temp_Ambient,CO2_Headspace,RH_Amb,Unnamed: 7
0,11/03/2020 05:51 PM,14.682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5559,06/23/2021 08:16 AM,14.138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5561,06/23/2021 10:16 AM,14.376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
result.to_csv(path / 'test.csv', index=False)